The first step to begin in a project is charge the libraries.

In [1]:
# This library works with DataFrames. A DataFrame treats the data as a table, like a sheet in Excel.
import pandas as pd
# This library has models that we will use. 
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import SGDClassifier
from sklearn.tree import DecisionTreeClassifier

In [2]:
# Load the dataset
dataTrain = pd.read_csv("./input/train.csv")
dataTest = pd.read_csv("./input/test.csv")
# Combine both datasets
dataset = [dataTrain, dataTest]

In [3]:
# Show a summary of the data in Dataset
dataTrain.describe(include='all')

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
count,891.000000,891.000000,891.000000,891,891,714.000000,891.000000,891.000000,891,891.000000,204,889
unique,NaN,NaN,NaN,891,2,NaN,NaN,NaN,681,NaN,147,3
top,NaN,NaN,NaN,"Graham, Mr. George Edward",male,NaN,NaN,NaN,CA. 2343,NaN,C23 C25 C27,S
freq,NaN,NaN,NaN,1,577,NaN,NaN,NaN,7,NaN,4,644
mean,446.000000,0.383838,2.308642,NaN,NaN,29.699118,0.523008,0.381594,NaN,32.204208,NaN,NaN
std,257.353842,0.486592,0.836071,NaN,NaN,14.526497,1.102743,0.806057,NaN,49.693429,NaN,NaN
min,1.000000,0.000000,1.000000,NaN,NaN,0.420000,0.000000,0.000000,NaN,0.000000,NaN,NaN
25%,223.500000,0.000000,2.000000,NaN,NaN,20.125000,0.000000,0.000000,NaN,7.910400,NaN,NaN
50%,446.000000,0.000000,3.000000,NaN,NaN,28.000000,0.000000,0.000000,NaN,14.454200,NaN,NaN
75%,668.500000,1.000000,3.000000,NaN,NaN,38.000000,1.000000,0.000000,NaN,31.000000,NaN,NaN


We are going to intepret this data. We have the columns of the table.
count = size of not null (not missing) in the fields
mean = average
std = standard desviation
min, 25%, 50%, 75%, max = minimum, 25%, 50%, 75%, maximum

In [4]:
# Show some random table rows with all columns
dataTrain.sample(3)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
425,426,0,3,"Wiseman, Mr. Phillippe",male,NaN,0,0,A/4. 34244,7.2500,NaN,S
367,368,1,3,"Moussa, Mrs. (Mantoura Boulos)",female,NaN,0,0,2626,7.2292,NaN,C
375,376,1,1,"Meyer, Mrs. Edgar Joseph (Leila Saks)",female,NaN,1,0,PC 17604,82.1708,NaN,C


We choose the target of the problem and in Titanic's problem is find who survived and who not.

In [5]:
# Select single column
titanicSurvived = dataTrain.Survived
# Print this column with its column type
print(titanicSurvived.head())

0    0
1    1
2    1
3    1
4    0
Name: Survived, dtype: int64


In [6]:
# Select multiple columns
titanicSexAndName = ['Sex', 'Name']
twoColumns = dataTrain[titanicSexAndName]
# Print these columns with its columns types
twoColumns.head()

,Sex,Name
0,male,"Braund, Mr. Owen Harris"
1,female,"Cumings, Mrs. John Bradley (Florence Briggs Th..."
2,female,"Heikkinen, Miss. Laina"
3,female,"Futrelle, Mrs. Jacques Heath (Lily May Peel)"
4,male,"Allen, Mr. William Henry"


We can see that Cabin column has a lot of missing values, so It is a irrelevant column, we are going to drop it

In [7]:
# Drop Cabin and Ticket columns
dataTrain = dataTrain.drop(['Ticket', 'Cabin'], axis=1)
dataTest = dataTest.drop(['Ticket', 'Cabin'], axis=1)
# We combine again both datasets
dataset = [dataTrain, dataTest]

We check that we dropped it successful

In [8]:
print(dataTrain.columns)

Index([u'PassengerId', u'Survived', u'Pclass', u'Name', u'Sex', u'Age',
       u'SibSp', u'Parch', u'Fare', u'Embarked'],
      dtype='object')


We are going to clean some rows like in column 'Name' because we have a lot of different titles

In [9]:
for each in dataset:
    each['Title'] = each.Name.str.extract(' ([A-Za-z]+)\.', expand=False)
# We see how we have a lot of different titles, we will change by others
pd.crosstab(dataTrain['Title'], dataTrain['Sex'])

Sex,female,male
Title,,
Capt,0,1
Col,0,2
Countess,1,0
Don,0,1
Dr,1,6
Jonkheer,0,1
Lady,1,0
Major,0,2
Master,0,40


We see how there are too much different titles

In [10]:
# We replace some titles in few titles
for each in dataset:
    each['Title'] = each['Title'].replace(['Lady','Capt', 'Countess', 'Col', 'Don', 'Dr', 'Major', 'Rev', 'Sir', 'Jonkheer', 'Dona'], 'Other')
    each['Title'] = each['Title'].replace('Mlle', 'Miss')
    each['Title'] = each['Title'].replace('Ms', 'Miss')
    each['Title'] = each['Title'].replace('Mme', 'Mrs')
# We calculate mean to see the titles
dataTrain[['Title', 'Survived']].groupby(['Title'], as_index=False).mean()

,Title,Survived
0,Master,0.575000
1,Miss,0.702703
2,Mr,0.156673
3,Mrs,0.793651
4,Other,0.347826


We are going to change categorical information to cardinal information

In [11]:
title = {"Master":1, "Miss":2, "Mr":3, "Mrs":4, "Other":5}
for each in dataset:
    each['Title'] = each['Title'].map(title)
    each['Title'] = each['Title'].fillna(0)
dataTrain.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Fare,Embarked,Title
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,7.2500,S,3
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,71.2833,C,4
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,7.9250,S,2
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,53.1000,S,4
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,8.0500,S,3


We can drop Name, Age, Fare and PassengerId columns because they are not relevant. Further we will change all categorical data in numerical data

In [12]:
dataTrain = dataTrain.drop(['Name', 'PassengerId', 'Age', 'Fare'], axis=1)
dataTest = dataTest.drop(['Name', 'Age', 'Fare'], axis=1)
dataset = [dataTrain, dataTest]

In [13]:
for each in dataset:
    each['Sex'] = each['Sex'].map({'female':0, 'male':1}).astype(int)
dataTrain.head()

,Survived,Pclass,Sex,SibSp,Parch,Embarked,Title
0,0,3,1,1,0,S,3
1,1,1,0,1,0,C,4
2,1,3,0,0,0,S,2
3,1,1,0,1,0,S,4
4,0,3,1,0,0,S,3


How Embarked column has two missing values, we are going to fill with the most common 

In [14]:
freqPort = dataTrain.Embarked.dropna().mode()[0]
freqPort

'S'

In [15]:
dataTest.describe(include='all')

,PassengerId,Pclass,Sex,SibSp,Parch,Embarked,Title
count,418.000000,418.000000,418.000000,418.000000,418.000000,418,418.000000
unique,NaN,NaN,NaN,NaN,NaN,3,NaN
top,NaN,NaN,NaN,NaN,NaN,S,NaN
freq,NaN,NaN,NaN,NaN,NaN,270,NaN
mean,1100.500000,2.265550,0.636364,0.447368,0.392344,NaN,2.911483
std,120.810458,0.841838,0.481622,0.896760,0.981429,NaN,0.783102
min,892.000000,1.000000,0.000000,0.000000,0.000000,NaN,1.000000
25%,996.250000,1.000000,0.000000,0.000000,0.000000,NaN,3.000000
50%,1100.500000,3.000000,1.000000,0.000000,0.000000,NaN,3.000000
75%,1204.750000,3.000000,1.000000,1.000000,0.000000,NaN,3.000000


In [16]:
for each in dataset:
    each['Embarked'] = each['Embarked'].fillna('S')
    each['Embarked'] = each['Embarked'].map({'S':0, 'Q':1, 'C':2}).astype(int)
dataTrain.head()

,Survived,Pclass,Sex,SibSp,Parch,Embarked,Title
0,0,3,1,1,0,0,3
1,1,1,0,1,0,2,4
2,1,3,0,0,0,0,2
3,1,1,0,1,0,0,4
4,0,3,1,0,0,0,3


In [17]:
# Parameters
x_dataTrain = dataTrain.drop('Survived', axis=1)
y_dataTrain = dataTrain["Survived"]
x_dataTest = dataTest.drop('PassengerId', axis=1).copy()

We have the criteria to create our model now. But we cannot fit the model because we have missing values, we need to fix it. 

We are going to choose some models, in this project the target is who survived in the accident

In [18]:
models = {
    'logReg': LogisticRegression(),
    'svc': SVC(),
    'knn': KNeighborsClassifier(n_neighbors = 3),
    'gauss': GaussianNB(),
    'sgd': SGDClassifier(),
    'tree': DecisionTreeClassifier(),
    'rf': RandomForestClassifier(n_estimators=100)
}

for id, cls in models.items():
    cls.fit(x_dataTrain, y_dataTrain)
    Y_predict = cls.predict(x_dataTest)
    acc_log = round(cls.score(x_dataTrain, y_dataTrain) * 100, 2)
    print(id, acc_log)

('knn', 84.29)
('tree', 85.52)
('svc', 83.61)
('gauss', 80.02)
('rf', 85.52)
('logReg', 79.57)
('sgd', 77.67)


/usr/local/lib/python2.7/dist-packages/sklearn/linear_model/stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)


In [20]:
# Stack submission
submission = pd.DataFrame({
        "PassengerId": dataTest["PassengerId"],
        "Survived": Y_predict
    })
submission.to_csv('./submission.csv', index=False)